<a href="https://colab.research.google.com/github/Edfred1/Contextual-Music-Crafter/blob/main/CMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation and Setup

This section covers cloning the repository and installing the necessary dependencies.

In [ ]:
# Clone the Git repository
!git clone https://github.com/Edfred1/Contextual-Music-Crafter.git
%cd Contextual-Music-Crafter

# Install dependencies
!pip install -r requirements.txt

### Configuration

Here, we configure the `config.yaml` file. This file contains settings for the AI API, musical parameters, and instrument definitions. You'll need to replace `"YOUR_GOOGLE_AI_API_KEY"` with your actual Google AI API key.

In [ ]:
import yaml

# Path to config.yaml
config_path = 'config.yaml'

new_config_content = """
# API Configuration
# Replace with your actual Google AI API key.
api_key: "YOUR_GOOGLE_AI_API_KEY" #YOUR_GOOGLE_AI_API_KEY
model_name: "gemini-2.5-pro" #"gemini-2.5-pro-exp-03-25"  #gemini-2.5-flash-preview-05-20 #gemini-2.5-flash-preview-04-17model_name: "gemini-1.5-flash" # Recommended model
# Temperature for the model (0.0 = deterministic, 2.0 = very creative). Default is 1.0.
temperature: 0.1

# --- Musical Parameters ---
# Use these settings to define the core musical direction of your generated track.

# A detailed description of the musical style you want to create.
# Be specific about feeling, instrumentation, and rhythm. You can also name
# artists (e.g. "in the style of Daft Punk"), songs, albums, or other concepts as inspiration.
# Example: "A high-energy, four-on-the-floor house track in the style of Daft Punk.
# Focus on a funky, filtered bassline, a classic house piano chord progression,
# and a simple, catchy lead synth melody. The drums should be punchy and groovy."
inspiration: "A high-energy, four-on-the-floor house track with a funky bassline."

# The primary genre of the track. This helps the AI understand the conventions to follow.
genre: "House Music"

# Beats Per Minute (Tempo) of the song.
bpm: 125

# The key and scale of the song (e.g., "C major", "A minor", "F# dorian").
# The script supports a wide variety of scales and church modes.
# Available scales: "major", "ionian", "minor", "natural minor", "aeolian",
# "harmonic minor", "melodic minor", "dorian", "phrygian", "lydian",
# "mixolydian", "locrian", "major pentatonic", "minor pentatonic", "chromatic",
# "whole tone", "diminished", "augmented", "byzantine", "hungarian minor",
# "persian", "arabic", "jewish", "ahava raba", "blues", "major blues"
key_scale: "C minor"

# Enables a "call and response" generation mode for melodic instruments.
# 0 = Disabled: Each track is generated independently based on the full context.
# 1 = Enabled: Melodic instruments will try to create a dialogue by playing in the gaps left by other melodic parts.
use_call_and_response: 0

# The number of complete songs to generate in a single run.
number_of_iterations: 1

# The time signature of the song.
time_signature:
  beats_per_bar: 4
  beat_value: 4 # (4 = quarter note)

# --- Instrument Definitions ---
# Define the instruments that will make up your arrangement.
# Each instrument is a list item with a `name`, a `program_num` (General MIDI program number), and a `role`.
# The `role` tells the AI the musical function of the track.
# Available Roles:
#   - "drums":      The main rhythmic foundation (kick, snare, hats).
#   - "percussion": Complementary rhythmic elements (bongos, congas, shakers).
#   - "sub_bass":   Very deep, fundamental bass (often a sine wave).
#   - "bass":       The main, rhythmic bassline. (Responds to Call/Response)
#   - "pads":       Atmospheric pads, long/slow chords.
#   - "atmosphere": Similar to pads, often more textured or evolving.
#   - "texture":    Sonic textures, not necessarily melodic or rhythmic.
#   - "chords":     Defines the chord progression, can be rhythmic stabs or sustained. (Responds to Call/Response)
#   - "harmony":    Similar to chords, providing harmonic structure.
#   - "arp":        An arpeggio; plays chord notes sequentially in a pattern. (Responds to Call/Response)
#   - "guitar":     Versatile for riffs, melodies, or rhythmic chords. (Responds to Call/Response)
#   - "lead":       The main melody or riff, the focal point. (Responds to Call/Response)
#   - "melody":     A secondary or supporting melody line. (Responds to Call/Response)
#   - "vocal":      Rhythmic vocal chops or short, catchy phrases. (Responds to Call/Response)
#   - "fx":         Sound effects, risers, impacts, sweeps, etc.
#   - "complementary": A general, complementary role (default if no role is specified).

#The order in this list determines the generation order.
# You can add as many instruments as you like to the list below.
# The script will generate a part for each one sequentially.
instruments:
  - name: "Drums"
    program_num: 10      # Standard Drum Kit (often ignored by DAWs, but required)
    role: "drums"

  - name: "Funky Bass"
    program_num: 39 # Synth Bass 2
    role: "bass"

  - name: "House Piano"
    program_num: 2  # Acoustic Grand Piano
    role: "harmony"

  - name: "Lead Synth"
    program_num: 82 # Lead 2 (sawtooth)
    role: "melody"

  - name: "Atmospheric Pads"
    program_num: 90 # Pad 2 (warm)
    role: "complementary"

# --- MIDI Technical Reference ---
# This section is for information only and does not configure the script.
#
# General MIDI Channel Assignments:
# The script automatically handles channel assignments based on instrument roles.
#
# - Channel 10 (internal index 9): Reserved for Drums & Percussion.
#   Any instrument with the role "drums" or "percussion" will be assigned to this channel
#   to ensure it plays with a drum kit sound in any standard MIDI player.
#
# - Channels 1-9 & 11-16: Used for Melodic & Tonal Instruments.
#   All other roles (bass, pads, lead, etc.) are assigned to these channels.
#   The script assigns them sequentially, skipping channel 10.
#
# General MIDI Program Numbers (Examples for reference):
#   1-8: Pianos
#   9-16: Chromatic Percussion
#   17-24: Organs
#   25-32: Guitars
#   33-40: Basses
#   41-48: Strings
#   49-56: Ensemble
#   57-64: Brass
#   65-72: Reed
#   73-80: Pipe
#   81-88: Synth Lead
#   89-96: Synth Pad
#   97-104: Synth Effects
#   105-112: Ethnic
#   113-120: Percussive
#   121-128: Sound Effects

# ALL MIDI Program Numbers
#
# 1-8: Pianos
# 1. Acoustic Grand Piano
# 2. Bright Acoustic Piano
# 3. Electric Grand Piano
# 4. Honky-Tonk Piano
# 5. Electric Piano 1
# 6. Electric Piano 2
# 7. Harpsichord
# 8. Clavinet

# 9-16: Chromatic Percussion
# 9. Celesta
# 10. Glockenspiel
# 11. Music Box
# 12. Vibraphone
# 13. Marimba
# 14. Xylophone
# 15. Tubular Bells
# 16. Dulcimer

# 17-24: Organs
# 17. Drawbar Organ
# 18. Percussive Organ
# 19. Rock Organ
# 20. Church Organ
# 21. Reed Organ
# 22. Accordion
# 23. Harmonica
# 24. Tango Accordion

# 25-32: Guitars
# 25. Acoustic Guitar (nylon)
# 26. Acoustic Guitar (steel)
# 27. Electric Guitar (jazz)
# 28. Electric Guitar (clean)
# 29. Electric Guitar (muted)
# 30. Overdriven Guitar
# 31. Distortion Guitar
# 32. Guitar Harmonics

# 33-40: Basses
# 33. Acoustic Bass
# 34. Electric Bass (finger)
# 35. Electric Bass (pick)
# 36. Fretless Bass
# 37. Slap Bass 1
# 38. Slap Bass 2
# 39. Synth Bass 1
# 40. Synth Bass 2

# 41-48: Strings
# 41. Violin
# 42. Viola
# 43. Cello
# 44. Contrabass
# 45. Tremolo Strings
# 46. Pizzicato Strings
# 47. Orchestral Harp
# 48. Timpani

# 49-56: Ensemble
# 49. String Ensemble 1
# 50. String Ensemble 2
# 51. SynthStrings 1
# 52. SynthStrings 2
# 53. Choir Aahs
# 54. Voice Oohs
# 55. Synth Voice
# 56. Orchestra Hit

# 57-64: Brass
# 57. Trumpet
# 58. Trombone
# 59. Tuba
# 60. Muted Trumpet
# 61. French Horn
# 62. Brass Section
# 63. Synth Brass 1
# 64. Synth Brass 2

# 65-72: Reed
# 65. Soprano Sax
# 66. Alto Sax
# 67. Tenor Sax
# 68. Baritone Sax
# 69. Oboe
# 70. English Horn
# 71. Bassoon
# 72. Clarinet

# 73-80: Pipe
# 73. Piccolo
# 74. Flute
# 75. Recorder
# 76. Pan Flute
# 77. Blown Bottle
# 78. Shakuhachi
# 79. Whistle
# 80. Ocarina

# 81-88: Synth Lead
# 81. Lead 1 (square)
# 82. Lead 2 (sawtooth)
# 83. Lead 3 (calliope)
# 84. Lead 4 (chiff)
# 85. Lead 5 (charang)
# 86. Lead 6 (voice)
# 87. Lead 7 (fifths)
# 88. Lead 8 (bass + lead)

# 89-96: Synth Pad
# 89. Pad 1 (new age)
# 90. Pad 2 (warm)
# 91. Pad 3 (polysynth)
# 92. Pad 4 (choir)
# 93. Pad 5 (bowed)
# 94. Pad 6 (metallic)
# 95. Pad 7 (halo)
# 96. Pad 8 (sweep)

# 97-104: Synth Effects
# 97. FX 1 (rain)
# 98. FX 2 (soundtrack)
# 99. FX 3 (crystal)
# 100. FX 4 (atmosphere)
# 101. FX 5 (brightness)
# 102. FX 6 (goblins)
# 103. FX 7 (echoes)
# 104. FX 8 (sci-fi)

# 105-112: Ethnic
# 105. Sitar
# 106. Banjo
# 107. Shamisen
# 108. Koto
# 109. Kalimba
# 110. Bagpipe
# 111. Fiddle
# 112. Shanai

# 113-120: Percussive
# 113. Tinkle Bell
# 114. Agogo
# 115. Steel Drums
# 116. Woodblock
# 117. Taiko Drum
# 118. Melodic Tom
# 119. Synth Drum
# 120. Reverse Cymbal

# 121-128: Sound Effects
# 121. Guitar Fret Noise
# 122. Breath Noise
# 123. Seashore
# 124. Bird Tweet
# 125. Telephone Ring
# 126. Helicopter
# 127. Applause
# 128. Gunshot
"""

# Write the new content to config.yaml
with open(config_path, 'w') as file:
    file.write(new_config_content)

print(f"Configuration updated in {config_path}.")

# Optional: Display the updated content to verify the change
with open(config_path, 'r') as file:
    print("\nUpdated content of config.yaml:")
    print(file.read())

### Running a Python Program from the Repository

This cell lists the Python programs found in the cloned repository and prompts you to select one to execute. The selected program will then be run based on the configuration.

In [ ]:
import os

# Get a list of all files and directories in the cloned repository
files_and_directories = os.listdir('.')

# Filter for Python files
python_files = [f for f in files_and_directories if f.endswith('.py')]

if not python_files:
    print("No Python files found in the repository.")
else:
    print("Found Python files:")
    for i, filename in enumerate(python_files):
        print(f"{i+1}. {filename}")

    # Prompt the user to select a file
    while True:
        try:
            choice = int(input(f"Enter the number of the file to execute (1-{len(python_files)}): "))
            if 1 <= choice <= len(python_files):
                selected_file = python_files[choice - 1]
                print(f"Executing {selected_file}...")
                # Execute the selected Python file
                !python {selected_file}
                break
            else:
                print("Invalid input. Please enter a number from the list.")
        except ValueError:
            print("Invalid input. Please enter a number.")